Preparing the R session


In [ ]:
source("install.R")

At first we load the data for our short experiment:

In [ ]:
# Read data for UK and Austria
uk_ten <- readRDS("data/ESS10.Rdata") %>%
  dplyr::filter(cntry == "GB") 
at_ten <- readRDS("data/ESS10SC.Rdata") %>%
  dplyr::filter(cntry == "AT")


In our second step we select the variables of interest:
Those are the ID numbers of the cases, the information on where (Country) the data was collected and then 5 variables that include different assessements. 

You can look up information on the data via the ESS homepage:
https://stessrelpubprodwe.blob.core.windows.net/data/round10/fieldwork/source/ESS10_source_questionnaires.pdf
https://stessrelpubprodwe.blob.core.windows.net/data/round10/fieldwork/source/ESS10_source_questionnaire_self_completion_paper.pdf

The codebook is online as well: https://ess-search.nsd.no/en/study/172ac431-2a06-41df-9dab-c1fd8f3877e7

In [ ]:
# Define variables of interest
Var_Selection <- c("idno", "cntry",
                   "gvhanc19", 
                   "gvjobc19", "gveldc19", 
                   "gvfamc19", "hscopc19" 
                   )

# Combine both datasets and select variables of interest
dataWS <- rbind(uk_ten[, Var_Selection], 
                at_ten[, Var_Selection])

# Print the merged dataset
print(dataWS)

# Create a table of counts for countries "AT" and "UK" to verify everything worked correctly x
country_counts <- table(dataWS$cntry)

table(dataWS$gvhanc19,
      dataWS$cntry)

Now we try to get info on response styles.
This is a first experiment. It is simply creating information on the amount of times someone agreed to a statement or used only the extreme categories at the far ends of a scale or only selected the same answers on all the items. 

In [ ]:




# count "extremes" #
lapply(Var_Selection[-c(1,2)], 
       function(VarName){
         x <- dataWS[VarName]
         return(sum(x == 0 | x == 10))
       }) %>%
  setNames(Var_Selection[-c(1,2)])


# Count "yes"
lapply(Var_Selection[-c(1,2)], 
       function(VarName){
         x <- dataWS[VarName]
         return(sum(x == 10))
       }) %>%
  setNames(Var_Selection[-c(1,2)])


# count "extremes" #
dataWS <- lapply(1:nrow(dataWS), function(RowNumber){
  x <- dataWS[RowNumber,Var_Selection[-c(1,2)]]
  return(sum(x == 0 | x == 10))
}) %>%
  unlist() %>%
  cbind(dataWS,.) %>%
  setNames(c(colnames(.)[-ncol(.)], "ExtremeValues"))

# Count "yes"
dataWS <- lapply(1:nrow(dataWS), function(RowNumber){
  x <- dataWS[RowNumber,Var_Selection[-c(1,2)]]
  return(sum(x == 10))
}) %>%
  unlist() %>%
  cbind(dataWS,.) %>%
  setNames(c(colnames(.)[-ncol(.)], "YesValues"))
  
# Categorize extremes 
dataWS$ExtremeCat[dataWS$ExtremeValues<3] <- 0
dataWS$ExtremeCat[dataWS$ExtremeValues==3] <- 1
dataWS$ExtremeCat[dataWS$ExtremeValues==4] <- 2
dataWS$ExtremeCat[dataWS$ExtremeValues==5] <- 3
dataWS$ExtremeCat <-  factor(dataWS$ExtremeCat, 
                             levels = 0:3,
                             labels = c("no tendency", "some tendency", 
                                        "tendency", "complete tendency"),
                             ordered = TRUE)

# Categorize yes
dataWS$YesCat[dataWS$YesValues<3] <- 0
dataWS$YesCat[dataWS$YesValues==3] <- 1
dataWS$YesCat[dataWS$YesValues==4] <- 2
dataWS$YesCat[dataWS$YesValues==5] <- 3
dataWS$YesCat <-  factor(dataWS$YesCat, 
                             levels = 0:3,
                             labels = c("no tendency", "some tendency", 
                                        "tendency", "complete tendency"),
                             ordered = TRUE)

## function to compute sum / mean scores with .X function ##
dataWS <- lapply(dataWS$idno, function(RowNumber){
  tmp <- dataWS[dataWS$idno == RowNumber,Var_Selection[-c(1,2)]]  
  return(min(tmp) == max(tmp))
}) %>%
  unlist() %>%
  cbind(dataWS,.) %>%
  setNames(c(colnames(.)[-ncol(.)], "Straight"))

dataWS %>%
  dplyr::select(ExtremeCat, YesCat, Straight) %>%
  DT::datatable()

table(dataWS$ExtremeCat,
      dataWS$cntry) %>%
  prop.table(2) %>%
  round(3)


table(dataWS$YesCat,
      dataWS$cntry) %>%
  prop.table(2) %>%
  round(3)


table(dataWS$Straight,
      dataWS$cntry) %>%
  prop.table(2) %>%
  round(3)

wilcox.test(as.numeric(ExtremeCat)~cntry, dataWS)

wilcox.test(as.numeric(YesCat)~cntry, dataWS)

wilcox.test(as.numeric(Straight)~cntry, dataWS)

